# Compile Probe Annotations

Got to do this in rpy2

In [ ]:
HDFS_DIR = '/cellar/users/agross/Data/SSD/'

In [1]:
import pandas as pd

In [38]:
from pandas.rpy.common import convert_robj
import rpy2.robjects as robjects
from pandas.rpy.common import convert_to_r_dataframe, convert_robj

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
require('IlluminaHumanMethylation450kanno.ilmn12.hg19')
ann = IlluminaHumanMethylation450kanno.ilmn12.hg19
data = ann@data

Loading required package: IlluminaHumanMethylation450kanno.ilmn12.hg19
Loading required package: minfi
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    duplicated, eval, evalq, Filter, Find, get, intersect, is.unsorted,
    lapply, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rep.int, rownames,
    sapply, setdiff, sort, table, tapply, union, unique, unlist

Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introd

In [8]:
islands = convert_robj(robjects.r("data.frame(data[['Islands.UCSC']])"))
locations = convert_robj(robjects.r("data.frame(data[['Locations']])"))
other = convert_robj(robjects.r("data.frame(data[['Other']])"))
other = other.ix[:, 2:] #first two columns are probe sequences which take up a lot of space
snps = convert_robj(robjects.r("data.frame(data[['SNPs.Illumina']])"))

In [9]:
PATH = '/cellar/users/agross/Data/GeneSets/methylation450_annotations.csv'
probe_annotations = pd.read_csv(PATH, index_col=0)

In [26]:
store = pd.HDFStore(HDFS_DIR + 'methylation_annotation.h5')
store.append('islands', islands)
store.create_table_index('islands', optlevel=9, kind='full')
store.append('locations', locations)
store.create_table_index('locations', optlevel=9, kind='full')
store.append('other', other)
store.create_table_index('other', optlevel=9, kind='full')
store.append('snps', snps)
store.create_table_index('snps', optlevel=9, kind='full')
store.append('probe_annotations', probe_annotations)
store.create_table_index('probe_annotations', optlevel=9, kind='full')

### Read in Flow Sorted Data and Convert to More Usable Format

In [27]:
%%R
require(minfi)
require(minfiData)
require(doParallel)

referencePkg <- "FlowSorted.Blood.450k"
require(referencePkg, character.only = TRUE)
data(list = referencePkg)
referenceRGset <- get(referencePkg)

Loading required package: minfiData
Loading required package: IlluminaHumanMethylation450kmanifest
Loading required package: doParallel
Loading required package: FlowSorted.Blood.450k


In [28]:
%%R
Mset.quantile = preprocessQuantile(referenceRGset, removeBadSamples = TRUE)
beta = getBeta(Mset.quantile)

[preprocessQuantile] Mapping to genome.
[preprocessQuantile] Fixing outliers.
[preprocessQuantile] Quantile normalizing.


In [29]:
beta = convert_robj(robjects.r("data.frame(beta)"))

In [30]:
labels = convert_robj(robjects.r('referenceRGset$CellType'))
label_map = pd.Series(labels, beta.columns)

In [31]:
store['label_map'] = label_map

In [ ]:
beta.to_hdf(store,'flow_sorted_data')

In [33]:
store.close()

In [34]:
robjects.r.library('WGCNA');
robjects.r.source("/cellar/users/agross/Data/MethylationAge/Horvath/NORMALIZATION.R");

Loading required package: dynamicTreeCut
Loading required package: flashClust

Attaching package: ‘flashClust’

The following object is masked from ‘package:stats’:

    hclust

*
*  Package WGCNA 1.41.1 loaded.
*
*    Important note: It appears that your system supports multi-threading,
*    but it is not enabled within WGCNA in R. 
*    To allow multi-threading within WGCNA with all available cores, use 
*
*          allowWGCNAThreads()
*
*    within R. Use disableWGCNAThreads() to disable threading if necessary.
*    Alternatively, set the following environment variable on your system:
*
*          ALLOW_WGCNA_THREADS=<number_of_processors>
*
*    for example 
*
*          ALLOW_WGCNA_THREADS=8
*
*    To set the environment variable in linux bash shell, type 
*
*           export ALLOW_WGCNA_THREADS=8
*
*     before running R. Other operating systems or shells will
*     have a similar command to achieve the same aim.
*



Attaching package: ‘WGCNA’

The following object is masked f

In [35]:
gold_standard = pd.read_csv('/cellar/users/agross/Data/MethylationAge/Horvath/probeAnnotation21kdatMethUsed.csv', 
                            index_col=0)

In [39]:
def normalize_meth(df):
    df = df.ix[gold_standard.index]
    df = df.T.fillna(gold_standard.goldstandard2).T

    df_r = robjects.r.t(convert_to_r_dataframe(df))
    gs = list(gold_standard.goldstandard2.ix[df.index])
    gs_r = robjects.FloatVector(gs)
    data_n_r = robjects.r.BMIQcalibration(df_r, gs_r)
    data_n = convert_robj(data_n_r).T
    return data_n

In [40]:
data_n = normalize_meth(beta)

[1] "Fitting EM beta mixture to goldstandard probes"
[1] Inf
[1] 0.008547826
[1] 0.007005981
[1] 0.009442499
[1] 0.01056228
[1] "Done"
ii= 1
[1] "Fitting EM beta mixture to input probes"
[1] Inf
[1] 0.01023027
[1] 0.009826291
[1] 0.009266958
[1] 0.008553946
[1] "Done"
[1] "Start normalising input probes"
ii= 2
[1] "Fitting EM beta mixture to input probes"
[1] Inf
[1] 0.009436482
[1] 0.009010343
[1] 0.008323679
[1] 0.007523282
[1] "Done"
[1] "Start normalising input probes"
ii= 3
[1] "Fitting EM beta mixture to input probes"
[1] Inf
[1] 0.01068228
[1] 0.01005551
[1] 0.009282203
[1] 0.008314943
[1] "Done"
[1] "Start normalising input probes"
ii= 4
[1] "Fitting EM beta mixture to input probes"
[1] Inf
[1] 0.009733978
[1] 0.009811863
[1] 0.00962571
[1] 0.009045181
[1] "Done"
[1] "Start normalising input probes"
ii= 5
[1] "Fitting EM beta mixture to input probes"
[1] Inf
[1] 0.009489052
[1] 0.009409222
[1] 0.008959802
[1] 0.008283887
[1] "Done"
[1] "Start normalising input probes"
ii= 6
[1]

In [42]:
data_n.to_hdf(HDFS_DIR, 'methylation_annotation.h5',
              'flow_sorted_data_horvath_norm')